Carregando o dataset

In [1]:
import os
import pandas as pd
import numpy as np

dataset = pd.read_pickle(r'C:\Users\mathe\OneDrive\Área de Trabalho\SoccerIA\MathIA_v7\dataset_173cols_europeu.pkl')

nan_counts = dataset.isna().sum()
nan_tot = nan_counts.sum()
print(nan_tot)
print(dataset.shape)

0
(66840, 173)


Importando os dataframes de 2023

In [2]:
import os
import pandas as pd
import numpy as np

# Caminho para a pasta que contém os arquivos .xlsx de 2023
path = r'C:\Users\mathe\OneDrive\Área de Trabalho\SoccerIA\Planilhas2023'

# Dicionário para armazenar os dataframes
dataframes = {}
dfss = []
# Lista todos os arquivos na pasta
files = os.listdir(path)

# Filtra a lista de arquivos para incluir apenas os arquivos .xlsx
xlsx_files = [f for f in files if f.endswith('.xlsx')]

# Carrega cada arquivo .xlsx em um dataframe e armazena no dicionário
for file in xlsx_files:
    full_path = os.path.join(path, file)  # junta o caminho do diretório com o nome do arquivo
    dataframes[file] = pd.read_excel(full_path)  # lê o arquivo .xlsx do caminho completo

    if "Unnamed: 0" in dataframes[file].columns:
        dataframes[file].drop("Unnamed: 0", axis=1, inplace=True)

    if "match_report_url" in dataframes[file].columns:
        dataframes[file].drop("match_report_url", axis=1, inplace=True)    

    if "team1_yellow_cards" in dataframes[file].columns:
        dataframes[file].drop("team1_yellow_cards", axis=1, inplace=True)

    if "team2_yellow_cards" in dataframes[file].columns:
        dataframes[file].drop("team2_yellow_cards", axis=1, inplace=True)  
    dfss.append(dataframes[file])

    print(file)
    print(dataframes[file].shape)
    nan_counts = dataframes[file].isna().sum()
    print(nan_counts)


# Combine all the DataFrames into a single DataFrame
combined_df_2023 = pd.concat(dfss, ignore_index=True)


combined_df_2023['team1'] = combined_df_2023['team1'].str.lower()
combined_df_2023['team2'] = combined_df_2023['team2'].str.lower()
combined_df_2023.replace('', np.nan, inplace=True)
# Add a column to mark future matches
combined_df_2023['is_future_match'] = combined_df_2023['team1_goals'].isna() | combined_df_2023['team2_goals'].isna()
combined_df_2023['season'] = '2023'
# Replace empty string with NaN
combined_df_2023["team1_red_cards"].replace('', np.nan, inplace=True)
combined_df_2023["team2_red_cards"].replace('', np.nan, inplace=True)

# Replace NaN with 0
combined_df_2023["team1_red_cards"].fillna(0, inplace=True)
combined_df_2023["team2_red_cards"].fillna(0, inplace=True)

combined_df_2023['date'] = pd.to_datetime(combined_df_2023['date'], format='%Y-%m-%d', errors='coerce')
combined_df_2023.sort_values('date', inplace=True)

print(combined_df_2023.columns)
print(combined_df_2023.shape) 


2023_ALE_B.xlsx
(90, 19)
team1                     0
team2                     0
team1_goals              36
team2_goals              36
season                    0
championship              0
team1_shots_on_target    36
team1_shots_out          36
team2_shots_on_target    36
team2_shots_out          36
team1_red_cards          36
team2_red_cards          36
team1_fouls              36
team2_fouls              36
team1_corners            36
team2_corners            36
team1_total_shots        36
team2_total_shots        36
date                      0
dtype: int64
2023_ALE_C.xlsx
(120, 19)
team1                     0
team2                     0
team1_goals              60
team2_goals              60
season                    0
championship              0
team1_shots_on_target    60
team1_shots_out          60
team2_shots_on_target    60
team2_shots_out          60
team1_red_cards          60
team2_red_cards          60
team1_fouls              60
team2_fouls              60
team1_corner

In [3]:
import pandas as pd
import numpy as np


combined_df_2023.sort_values('date', inplace=True)


combined_df_2023['team1_goals'] = pd.to_numeric(combined_df_2023['team1_goals'], errors='coerce')
combined_df_2023['team2_goals'] = pd.to_numeric(combined_df_2023['team2_goals'], errors='coerce')

# calculate goal differences
combined_df_2023['goal_diff_team1'] = combined_df_2023['team1_goals'] - combined_df_2023['team2_goals']
combined_df_2023['goal_diff_team2'] = combined_df_2023['team2_goals'] - combined_df_2023['team1_goals']

# calculate corners differences
combined_df_2023['corners_diff_team1'] = combined_df_2023['team1_corners'] - combined_df_2023['team2_corners']
combined_df_2023['corners_diff_team2'] = combined_df_2023['team2_corners'] - combined_df_2023['team1_corners']

# calculate big wins and losses
combined_df_2023['team1_big_win'] = np.where(combined_df_2023['goal_diff_team1'] >= 2, 1, 0)
combined_df_2023['team1_big_loss'] = np.where(combined_df_2023['goal_diff_team1'] <= -2, 1, 0)
combined_df_2023['team2_big_win'] = np.where(combined_df_2023['goal_diff_team2'] >= 2, 1, 0)
combined_df_2023['team2_big_loss'] = np.where(combined_df_2023['goal_diff_team2'] <= -2, 1, 0)

# calculate AH-2.5 win and losses
combined_df_2023['team1_ah-2.5_win'] = np.where(combined_df_2023['corners_diff_team1'] >= 3, 1, 0)
combined_df_2023['team1_ah-2.5_loss'] = np.where(combined_df_2023['corners_diff_team1'] <= 2, 1, 0)
combined_df_2023['team2_ah-2.5_win'] = np.where(combined_df_2023['corners_diff_team2'] >= 3, 1, 0)
combined_df_2023['team2_ah-2.5_loss'] = np.where(combined_df_2023['corners_diff_team2'] <= 2, 1, 0)


# calculate AH+2.5 win and losses
combined_df_2023['team1_ah+2.5_win'] = np.where(combined_df_2023['corners_diff_team1'] >= -2, 1, 0)
combined_df_2023['team1_ah+2.5_loss'] = np.where(combined_df_2023['corners_diff_team1'] <= -3, 1, 0)
combined_df_2023['team2_ah+2.5_win'] = np.where(combined_df_2023['corners_diff_team2'] >= -2, 1, 0)
combined_df_2023['team2_ah+2.5_loss'] = np.where(combined_df_2023['corners_diff_team2'] <= -3, 1, 0)


# calculate over4.5 win and losses
combined_df_2023['team1_over4.5'] = np.where(combined_df_2023['team1_corners'] >= 5, 1, 0)
combined_df_2023['team1_under4.5'] = np.where(combined_df_2023['team1_corners'] <= 4, 1, 0)
combined_df_2023['team2_over4.5'] = np.where(combined_df_2023['team2_corners'] >= 5, 1, 0)
combined_df_2023['team2_under4.5'] = np.where(combined_df_2023['team2_corners'] <= 4, 1, 0)

# calculate over3.5 win and losses
combined_df_2023['team1_over3.5'] = np.where(combined_df_2023['team1_corners'] >= 4, 1, 0)
combined_df_2023['team1_under3.5'] = np.where(combined_df_2023['team1_corners'] <= 3, 1, 0)
combined_df_2023['team2_over3.5'] = np.where(combined_df_2023['team2_corners'] >= 4, 1, 0)
combined_df_2023['team2_under3.5'] = np.where(combined_df_2023['team2_corners'] <= 3, 1, 0)

# calculate over6.5 win and losses
combined_df_2023['team1_over6.5'] = np.where(combined_df_2023['team1_corners'] >= 7, 1, 0)
combined_df_2023['team1_under6.5'] = np.where(combined_df_2023['team1_corners'] <= 6, 1, 0)
combined_df_2023['team2_over6.5'] = np.where(combined_df_2023['team2_corners'] >= 7, 1, 0)
combined_df_2023['team2_under6.5'] = np.where(combined_df_2023['team2_corners'] <= 6, 1, 0)



# Initialize these columns with 0
combined_df_2023['team1_big_wins_last5'] = 0
combined_df_2023['team1_big_losses_last5'] = 0
combined_df_2023['team2_big_wins_last5'] = 0
combined_df_2023['team2_big_losses_last5'] = 0

combined_df_2023['team1_ah-2.5_wins_last5'] = 0 
combined_df_2023['team1_ah-2.5_losses_last5'] = 0 
combined_df_2023['team2_ah-2.5_wins_last5'] = 0 
combined_df_2023['team2_ah-2.5_losses_last5'] = 0 

combined_df_2023['team1_ah+2.5_wins_last5'] = 0 
combined_df_2023['team1_ah+2.5_losses_last5'] = 0 
combined_df_2023['team2_ah+2.5_wins_last5'] = 0 
combined_df_2023['team2_ah+2.5_losses_last5'] = 0 

combined_df_2023['team1_over3.5_last5'] = 0 
combined_df_2023['team1_under3.5_last5'] = 0 
combined_df_2023['team2_over3.5_last5'] = 0 
combined_df_2023['team2_under3.5_last5'] = 0 

combined_df_2023['team1_over4.5_last5'] = 0 
combined_df_2023['team1_under4.5_last5'] = 0 
combined_df_2023['team2_over4.5_last5'] = 0
combined_df_2023['team2_under4.5_last5'] = 0

combined_df_2023['team1_over6.5_last5'] = 0
combined_df_2023['team1_under6.5_last5'] = 0
combined_df_2023['team2_over6.5_last5'] = 0
combined_df_2023['team2_under6.5_last5'] = 0


new_cols = ['avg_scr_lasts3_1_home', 'avg_scr_lasts5_1_home', 'avg_scr_lasts3_1_away',
            'avg_scr_lasts5_1_away', 'avg_conc_lasts3_1_home', 'avg_conc_lasts5_1_home',
            'avg_conc_lasts3_1_away', 'avg_conc_lasts5_1_away', 'avg_scr_lasts3_2_home',
            'avg_scr_lasts5_2_home', 'avg_scr_lasts3_2_away', 'avg_scr_lasts5_2_away',
            'avg_conc_lasts3_2_home', 'avg_conc_lasts5_2_home', 'avg_conc_lasts3_2_away',
            'avg_conc_lasts5_2_away','team1_big_wins_last5', 'team1_big_losses_last5', 
            'team2_big_wins_last5', 'team2_big_losses_last5',
            #abaixo vai ser baseado em finalizações
            'avg_total_shots_lasts5_1_home','avg_total_shots_lasts5_1_away','avg_total_shots_lasts5_2_home',
            'avg_total_shots_lasts5_2_away', 'avg_otarget_shots_lasts5_1_home','avg_otarget_shots_lasts5_1_away',
            'avg_otarget_shots_lasts5_2_home','avg_otarget_shots_lasts5_2_away','avg_out_shots_lasts5_1_home',
            'avg_out_shots_lasts5_1_away','avg_out_shots_lasts5_2_home','avg_out_shots_lasts5_2_away',
            'avg_conc_total_shots_lasts5_1_home','avg_conc_total_shots_lasts5_1_away',
            'avg_conc_total_shots_lasts5_2_home','avg_conc_total_shots_lasts5_2_away',
            #abaixo vai ser baseado em corners
            'avg_corners_lasts5_1_home','avg_corners_lasts5_1_away', 
            'avg_corners_conc_lasts5_1_home','avg_corners_conc_lasts5_1_away',
            'avg_corners_lasts5_2_home','avg_corners_lasts5_2_away', 
            'avg_corners_conc_lasts5_2_home', 'avg_corners_conc_lasts5_2_away',
            #abaixo vai ser baseado em fouls
            'avg_fouls_lasts5_1_home','avg_fouls_lasts5_1_away', 
            'avg_fouls_conc_lasts5_1_home', 'avg_fouls_conc_lasts5_1_away',
            'avg_fouls_lasts5_2_home','avg_fouls_lasts5_2_away', 
            'avg_fouls_conc_lasts5_2_home', 'avg_fouls_conc_lasts5_2_away',
            #novas colunas da v7
            'team1_ah-2.5_wins_last5', 'team1_ah-2.5_losses_last5','team2_ah-2.5_wins_last5','team2_ah-2.5_losses_last5',
            'team1_ah+2.5_wins_last5','team1_ah+2.5_losses_last5','team2_ah+2.5_wins_last5','team2_ah+2.5_losses_last5',
            'team1_over3.5_last5','team1_under3.5_last5','team2_over3.5_last5','team2_under3.5_last5',
            'team1_over4.5_last5','team1_under4.5_last5','team2_over4.5_last5','team2_under4.5_last5',
            'team1_over6.5_last5','team1_under6.5_last5','team2_over6.5_last5','team2_under6.5_last5',
            #v8
            'avg_corners_diff_lasts5_1_home', 'median_corners_diff_lasts5_1_home', 'avg_corners_diff_lasts5_1_away',
             'median_corners_diff_lasts5_1_away', 'avg_corners_diff_lasts5_2_home','median_corners_diff_lasts5_2_home', 
             'avg_corners_diff_lasts5_2_away','median_corners_diff_lasts5_2_away'
            ]

# Create a dictionary with keys as column names and values as np.nan
new_cols_dict = {col: np.nan for col in new_cols}

# Add new columns to the DataFrame
combined_df_2023 = combined_df_2023.assign(**new_cols_dict)

#ESTATÍSTICAS PARA O TIME, LEVANDO EM CONSIDERAÇÃO CASA E FORA, PORÉM É UMA ESTATÍSTICA ÚNICA PARA O CONFRONTRO
# Iterate over each row in the DataFrame
for i, row in combined_df_2023.iterrows():
    # For each team, get their past home and away matches before the current date
    team1_matches = combined_df_2023[((combined_df_2023['team1'] == row['team1']) | (combined_df_2023['team2'] == row['team1'])) & (combined_df_2023['date'] < row['date']) & (combined_df_2023['season'] == row['season'])].sort_values(by='date')
    team2_matches = combined_df_2023[((combined_df_2023['team1'] == row['team2']) | (combined_df_2023['team2'] == row['team2'])) & (combined_df_2023['date'] < row['date']) & (combined_df_2023['season'] == row['season'])].sort_values(by='date')

    # For each team, calculate stats for last 5 matches
    if not team1_matches.empty:
        team1_matches['big_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_big_win'], team1_matches['team2_big_win'])
        team1_matches['big_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_big_loss'], team1_matches['team2_big_loss'])
        combined_df_2023.at[i, 'team1_big_wins_last5'] = team1_matches['big_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_big_losses_last5'] = team1_matches['big_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para ah-2.5 para a equipe 1
        team1_matches['ah-2.5_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah-2.5_win'], team1_matches['team2_ah-2.5_win'])
        team1_matches['ah-2.5_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah-2.5_loss'], team1_matches['team2_ah-2.5_loss'])
        combined_df_2023.at[i, 'team1_ah-2.5_wins_last5'] = team1_matches['ah-2.5_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_ah-2.5_losses_last5'] = team1_matches['ah-2.5_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para ah+2.5 para a equipe 1
        team1_matches['ah+2.5_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah+2.5_win'], team1_matches['team2_ah+2.5_win'])
        team1_matches['ah+2.5_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah+2.5_loss'], team1_matches['team2_ah+2.5_loss'])
        combined_df_2023.at[i, 'team1_ah+2.5_wins_last5'] = team1_matches['ah+2.5_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_ah+2.5_losses_last5'] = team1_matches['ah+2.5_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over3.5 para a equipe 1
        team1_matches['over3.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over3.5'], team1_matches['team2_over3.5'])
        team1_matches['under3.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under3.5'], team1_matches['team2_under3.5'])
        combined_df_2023.at[i, 'team1_over3.5_last5'] = team1_matches['over3.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under3.5_last5'] = team1_matches['under3.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over4.5 para a equipe 1
        team1_matches['over4.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over4.5'], team1_matches['team2_over4.5'])
        team1_matches['under4.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under4.5'], team1_matches['team2_under4.5'])
        combined_df_2023.at[i, 'team1_over4.5_last5'] = team1_matches['over4.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under4.5_last5'] = team1_matches['under4.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over6.5 para a equipe 1
        team1_matches['over6.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over6.5'], team1_matches['team2_over6.5'])
        team1_matches['under6.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under6.5'], team1_matches['team2_under6.5'])
        combined_df_2023.at[i, 'team1_over6.5_last5'] = team1_matches['over6.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under6.5_last5'] = team1_matches['under6.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan



    if not team2_matches.empty:
        team2_matches['big_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_big_win'], team2_matches['team2_big_win'])
        team2_matches['big_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_big_loss'], team2_matches['team2_big_loss'])
        combined_df_2023.at[i, 'team2_big_wins_last5'] = team2_matches['big_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_big_losses_last5'] = team2_matches['big_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para ah-2.5 para a equipe 2
        team2_matches['ah-2.5_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah-2.5_win'], team2_matches['team2_ah-2.5_win'])
        team2_matches['ah-2.5_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah-2.5_loss'], team2_matches['team2_ah-2.5_loss'])
        combined_df_2023.at[i, 'team2_ah-2.5_wins_last5'] = team2_matches['ah-2.5_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_ah-2.5_losses_last5'] = team2_matches['ah-2.5_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para ah+2.5 para a equipe 2
        team2_matches['ah+2.5_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah+2.5_win'], team2_matches['team2_ah+2.5_win'])
        team2_matches['ah+2.5_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah+2.5_loss'], team2_matches['team2_ah+2.5_loss'])
        combined_df_2023.at[i, 'team2_ah+2.5_wins_last5'] = team2_matches['ah+2.5_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_ah+2.5_losses_last5'] = team2_matches['ah+2.5_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over3.5  para a equipe 2
        team2_matches['over3.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over3.5'], team2_matches['team2_over3.5'])
        team2_matches['under3.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under3.5'], team2_matches['team2_under3.5'])
        combined_df_2023.at[i, 'team2_over3.5_last5'] = team2_matches['over3.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under3.5_last5'] = team2_matches['under3.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over4.5  para a equipe 2
        team2_matches['over4.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over4.5'], team2_matches['team2_over4.5'])
        team2_matches['under4.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under4.5'], team2_matches['team2_under4.5'])
        combined_df_2023.at[i, 'team2_over4.5_last5'] = team2_matches['over4.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under4.5_last5'] = team2_matches['under4.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over6.5  para a equipe 2
        team2_matches['over6.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over6.5'], team2_matches['team2_over6.5'])
        team2_matches['under6.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under6.5'], team2_matches['team2_under6.5'])
        combined_df_2023.at[i, 'team2_over6.5_last5'] = team2_matches['over6.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under6.5_last5'] = team2_matches['under6.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan



 
#ESTATÍSTICAS PARA O TIME, LEVANDO EM CONSIDERAÇÃO CASA E FORA SEPARADAMENTE 
for i, row in combined_df_2023.iterrows():
    team1_home = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team1'] == row['team1']) & (combined_df_2023['season'] == row['season'])]
    team1_away = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team2'] == row['team1']) & (combined_df_2023['season'] == row['season'])]
    
    team2_home = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team1'] == row['team2']) & (combined_df_2023['season'] == row['season'])]
    team2_away = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team2'] == row['team2']) & (combined_df_2023['season'] == row['season'])]

    if not team1_home.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_1_home'] = team1_home['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_1_home'] = team1_home['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_1_home'] = team1_home['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_home['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_1_home'] = team1_home['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_1_home'] = team1_home['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_1_home'] = team1_home['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_1_home'] = team1_home['team1_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_1_home'] = team1_home['team1_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_shots_out'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_1_home'] = team1_home['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_1_home'] = team1_home['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_1_home'] = team1_home['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_1_home'] = team1_home['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_fouls'].isna().any() else np.nan
       
        # Mediana e Desvio Padrão para Finalizações (Shots)
        combined_df_2023.at[i, 'median_total_shots_lasts5_1_home'] = team1_home['team1_total_shots'].rolling(5, min_periods=2).median().iloc[-1] if not team1_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_total_shots_lasts5_1_home'] = team1_home['team1_total_shots'].rolling(5, min_periods=2).std().iloc[-1] if not team1_home['team1_total_shots'].isna().any() else np.nan
        
        # Mediana e Desvio Padrão para Corners
        combined_df_2023.at[i, 'median_corners_lasts5_1_home'] = team1_home['team1_corners'].rolling(5, min_periods=2).median().iloc[-1] if not team1_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_corners_lasts5_1_home'] = team1_home['team1_corners'].rolling(5, min_periods=2).std().iloc[-1] if not team1_home['team1_corners'].isna().any() else np.nan

        # Mediana e Desvio Padrão para Corners Concedidos
        combined_df_2023.at[i, 'median_conc_corners_lasts5_1_home'] = team1_home['team2_corners'].rolling(5, min_periods=2).median().iloc[-1] if not team1_home['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_conc_corners_lasts5_1_home'] = team1_home['team2_corners'].rolling(5, min_periods=2).std().iloc[-1] if not team1_home['team2_corners'].isna().any() else np.nan
        
        
        # Média e Mediana do saldo da diferença de escanteios para o time 1 nas últimas 5 partidas em casa
        combined_df_2023.at[i, 'avg_corners_diff_lasts5_1_home'] = team1_home['corners_diff_team1'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['corners_diff_team1'].isna().any() else np.nan
        combined_df_2023.at[i, 'median_corners_diff_lasts5_1_home'] = team1_home['corners_diff_team1'].rolling(5, min_periods=2).median().iloc[-1] if not team1_home['corners_diff_team1'].isna().any() else np.nan




    if not team1_away.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_1_away'] = team1_away['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_1_away'] = team1_away['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_1_away'] = team1_away['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_away['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_1_away'] = team1_away['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_1_away'] = team1_away['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_1_away'] = team1_away['team2_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_1_away'] = team1_away['team2_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_1_away'] = team1_away['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_1_away'] = team1_away['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_1_away'] = team1_away['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_1_away'] = team1_away['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_1_away'] = team1_away['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_fouls'].isna().any() else np.nan


        # Mediana e Desvio Padrão para Finalizações (Shots)
        combined_df_2023.at[i, 'median_total_shots_lasts5_1_away'] = team1_away['team2_total_shots'].rolling(5, min_periods=2).median().iloc[-1] if not team1_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_total_shots_lasts5_1_away'] = team1_away['team2_total_shots'].rolling(5, min_periods=2).std().iloc[-1] if not team1_away['team2_total_shots'].isna().any() else np.nan
        
        # Mediana e Desvio Padrão para Corners
        combined_df_2023.at[i, 'median_corners_lasts5_1_away'] = team1_away['team2_corners'].rolling(5, min_periods=2).median().iloc[-1] if not team1_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_corners_lasts5_1_away'] = team1_away['team2_corners'].rolling(5, min_periods=2).std().iloc[-1] if not team1_away['team2_corners'].isna().any() else np.nan
    
        # Mediana e Desvio Padrão para Corners Concedidos
        combined_df_2023.at[i, 'median_conc_corners_lasts5_1_away'] = team1_away['team1_corners'].rolling(5, min_periods=2).median().iloc[-1] if not team1_away['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_conc_corners_lasts5_1_away'] = team1_away['team1_corners'].rolling(5, min_periods=2).std().iloc[-1] if not team1_away['team1_corners'].isna().any() else np.nan


        # Média e Mediana do saldo da diferença de escanteios para o time 1 nas últimas 5 partidas fora de casa
        combined_df_2023.at[i, 'avg_corners_diff_lasts5_1_away'] = team1_away['corners_diff_team2'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['corners_diff_team2'].isna().any() else np.nan
        combined_df_2023.at[i, 'median_corners_diff_lasts5_1_away'] = team1_away['corners_diff_team2'].rolling(5, min_periods=2).median().iloc[-1] if not team1_away['corners_diff_team2'].isna().any() else np.nan


    if not team2_home.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_2_home'] = team2_home['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_2_home'] = team2_home['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_2_home'] = team2_home['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_home['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_2_home'] = team2_home['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_2_home'] = team2_home['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_2_home'] = team2_home['team1_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_2_home'] = team2_home['team1_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_2_home'] = team2_home['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_2_home'] = team2_home['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_2_home'] = team2_home['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_2_home'] = team2_home['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_2_home'] = team2_home['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_fouls'].isna().any() else np.nan

        # Mediana e Desvio Padrão para Finalizações (Shots)
        combined_df_2023.at[i, 'median_total_shots_lasts5_2_home'] = team2_home['team1_total_shots'].rolling(5, min_periods=2).median().iloc[-1] if not team2_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_total_shots_lasts5_2_home'] = team2_home['team1_total_shots'].rolling(5, min_periods=2).std().iloc[-1] if not team2_home['team1_total_shots'].isna().any() else np.nan
        
        # Mediana e Desvio Padrão para Corners
        combined_df_2023.at[i, 'median_corners_lasts5_2_home'] = team2_home['team1_corners'].rolling(5, min_periods=2).median().iloc[-1] if not team2_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_corners_lasts5_2_home'] = team2_home['team1_corners'].rolling(5, min_periods=2).std().iloc[-1] if not team2_home['team1_corners'].isna().any() else np.nan

        # Mediana e Desvio Padrão para Corners Concedidos
        combined_df_2023.at[i, 'median_conc_corners_lasts5_2_home'] = team2_home['team2_corners'].rolling(5, min_periods=2).median().iloc[-1] if not team2_home['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_conc_corners_lasts5_2_home'] = team2_home['team2_corners'].rolling(5, min_periods=2).std().iloc[-1] if not team2_home['team2_corners'].isna().any() else np.nan


        # Média e Mediana do saldo da diferença de escanteios para o time 2 nas últimas 5 partidas em casa
        combined_df_2023.at[i, 'avg_corners_diff_lasts5_2_home'] = team2_home['corners_diff_team2'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['corners_diff_team2'].isna().any() else np.nan
        combined_df_2023.at[i, 'median_corners_diff_lasts5_2_home'] = team2_home['corners_diff_team2'].rolling(5, min_periods=2).median().iloc[-1] if not team2_home['corners_diff_team2'].isna().any() else np.nan

    if not team2_away.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_2_away'] = team2_away['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_2_away'] = team2_away['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_2_away'] = team2_away['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_away['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_2_away'] = team2_away['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_2_away'] = team2_away['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_2_away'] = team2_away['team2_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_2_away'] = team2_away['team2_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_2_away'] = team2_away['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_2_away'] = team2_away['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_2_away'] = team2_away['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_2_away'] = team2_away['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_2_away'] = team2_away['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_fouls'].isna().any() else np.nan
        

        # Mediana e Desvio Padrão para Finalizações (Shots)
        combined_df_2023.at[i, 'median_total_shots_lasts5_2_away'] = team2_away['team2_total_shots'].rolling(5, min_periods=2).median().iloc[-1] if not team2_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_total_shots_lasts5_2_away'] = team2_away['team2_total_shots'].rolling(5, min_periods=2).std().iloc[-1] if not team2_away['team2_total_shots'].isna().any() else np.nan
        
        # Mediana e Desvio Padrão para Corners
        combined_df_2023.at[i, 'median_corners_lasts5_2_away'] = team2_away['team2_corners'].rolling(5, min_periods=2).median().iloc[-1] if not team2_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_corners_lasts5_2_away'] = team2_away['team2_corners'].rolling(5, min_periods=2).std().iloc[-1] if not team2_away['team2_corners'].isna().any() else np.nan

        # Mediana e Desvio Padrão para Corners Concedidos
        combined_df_2023.at[i, 'median_conc_corners_lasts5_2_away'] = team2_away['team1_corners'].rolling(5, min_periods=2).median().iloc[-1] if not team2_away['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'std_conc_corners_lasts5_2_away'] = team2_away['team1_corners'].rolling(5, min_periods=2).std().iloc[-1] if not team2_away['team1_corners'].isna().any() else np.nan


        # Média e Mediana do saldo da diferença de escanteios para o time 2 nas últimas 5 partidas fora de casa
        combined_df_2023.at[i, 'avg_corners_diff_lasts5_2_away'] = team2_away['corners_diff_team2'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['corners_diff_team2'].isna().any() else np.nan
        combined_df_2023.at[i, 'median_corners_diff_lasts5_2_away'] = team2_away['corners_diff_team2'].rolling(5, min_periods=2).median().iloc[-1] if not team2_away['corners_diff_team2'].isna().any() else np.nan


combined_df_2023.shape        

C:\Users\mathe\AppData\Local\Temp\ipykernel_13420\1722247661.py:249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df_2023.at[i, 'std_corners_lasts5_1_home'] = team1_home['team1_corners'].rolling(5, min_periods=2).std().iloc[-1] if not team1_home['team1_corners'].isna().any() else np.nan
C:\Users\mathe\AppData\Local\Temp\ipykernel_13420\1722247661.py:252: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df_2023.at[i, 'median_conc_corners_lasts5_1_home'] = team1_home['team2_corners'].rolling(5, min_periods=2).media

(1182, 152)

In [4]:
def get_result(row):
    if row['team1_goals'] > row['team2_goals']:
        return pd.Series([3, 0])
    elif row['team1_goals'] < row['team2_goals']:
        return pd.Series([0, 3])
    else:
        return pd.Series([1, 1])

combined_df_2023[['result_team1', 'result_team2']] = combined_df_2023.apply(get_result, axis=1)

def get_streak(df, result_col, results):
    result_series = df[result_col].apply(lambda x: 1 if x in results else 0)
    result_series = result_series * (result_series.groupby((result_series != result_series.shift()).cumsum()).cumcount() + 1)
    return result_series

# Create a dictionary to hold individual team dataframes
team_df_dict = {}

def get_individual_team_df(df, team_name): #teoricamente aqui deveria ser corners, mas o erro se apresentou menor assim:
    if team_name in team_df_dict:
        return team_df_dict[team_name]
        
    team_games = df[(df['team1'] == team_name) | (df['team2'] == team_name)].copy()
    team_games['team_is_team1'] = team_games['team1'] == team_name
    team_games['team_result'] = np.where(team_games['team_is_team1'], team_games['result_team1'], team_games['result_team2'])
    team_games['team_goals'] = np.where(team_games['team_is_team1'], team_games['team1_goals'], team_games['team2_goals'])
    team_games['team_redcards'] = np.where(team_games['team_is_team1'], team_games['team1_red_cards'], team_games['team2_red_cards'])

    team_games.sort_values('date', inplace=True)
    team_games['days_since_last_game'] = team_games['date'].diff().dt.days

    team_df_dict[team_name] = team_games
    return team_games

def get_team_stats(row, df):
    team1_games = get_individual_team_df(df, row['team1'])
    team2_games = get_individual_team_df(df, row['team2'])

    # Filter to include only games that occurred before the current game
    team1_games = team1_games[team1_games['date'] < row['date']]
    team2_games = team2_games[team2_games['date'] < row['date']]

    stats = {}

    if not team1_games.empty:
        stats['team1_winning_streak'] = get_streak(team1_games, 'team_result', [3]).iloc[-1]
        stats['team1_undefeated_streak'] = get_streak(team1_games, 'team_result', [1, 3]).iloc[-1]
        stats['team1_losing_streak'] = get_streak(team1_games, 'team_result', [0]).iloc[-1]
        stats['team1_without_winning_streak'] = get_streak(team1_games, 'team_result', [0, 1]).iloc[-1]
        stats['avg_points_lasts5_1'] = team1_games.tail(5)['team_result'].mean()
        stats['team1_strength'] = team1_games['team_goals'].sum() / (team1_games['team1_goals'].sum() + team1_games['team2_goals'].sum() + 0.01)
        stats['championship_points_1'] = team1_games['team_result'].sum() / len(team1_games)
        rested_4_or_more_days_1 = team1_games.tail(1)['days_since_last_game'].values[0] >= 4
        stats['rested_4_days_or_more_1'] = 1 if rested_4_or_more_days_1 else -1

    if not team2_games.empty:
        stats['team2_winning_streak'] = get_streak(team2_games, 'team_result', [3]).iloc[-1]
        stats['team2_undefeated_streak'] = get_streak(team2_games, 'team_result', [1, 3]).iloc[-1]
        stats['team2_losing_streak'] = get_streak(team2_games, 'team_result', [0]).iloc[-1]
        stats['team2_without_winning_streak'] = get_streak(team2_games, 'team_result', [0, 1]).iloc[-1]
        stats['avg_points_lasts5_2'] = team2_games.tail(5)['team_result'].mean()
        stats['team2_strength'] = team2_games['team_goals'].sum() / (team2_games['team1_goals'].sum() + team2_games['team2_goals'].sum() + 0.01)
        stats['championship_points_2'] = team2_games['team_result'].sum() / len(team2_games)
        rested_4_or_more_days_2 = team2_games.tail(1)['days_since_last_game'].values[0] >= 4
        stats['rested_4_days_or_more_2'] = 1 if rested_4_or_more_days_2 else -1

    return pd.Series(stats)

combined_df_2023 = pd.concat([combined_df_2023, combined_df_2023.apply(lambda row: get_team_stats(row, combined_df_2023), axis=1)], axis=1)

# Now, calculate the number of suspended players for the next match for each team.
for team_name in team_df_dict.keys():
    team_df = team_df_dict[team_name].copy()
    team_df['next_match_suspended_players'] = team_df['team_redcards'].shift()

    # Assign the suspended players back to the combined_df_2023.
    team1_mask = combined_df_2023['team1'] == team_name
    team2_mask = combined_df_2023['team2'] == team_name
    combined_df_2023.loc[team1_mask, 'team1_suspended_players'] = team_df.loc[team1_mask, 'next_match_suspended_players']
    combined_df_2023.loc[team2_mask, 'team2_suspended_players'] = team_df.loc[team2_mask, 'next_match_suspended_players']

combined_df_2023.shape    


C:\Users\mathe\AppData\Local\Temp\ipykernel_13420\1439903165.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df_2023[['result_team1', 'result_team2']] = combined_df_2023.apply(get_result, axis=1)
C:\Users\mathe\AppData\Local\Temp\ipykernel_13420\1439903165.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df_2023[['result_team1', 'result_team2']] = combined_df_2023.apply(get_result, axis=1)
C:\Users\mathe\AppData\Local\Temp\ipykernel_13420\1439903165.py:79: PerformanceWarning: DataFrame is highly fragment

(1182, 172)

In [5]:
list(combined_df_2023.columns)

['team1',
 'team2',
 'team1_goals',
 'team2_goals',
 'season',
 'championship',
 'team1_shots_on_target',
 'team1_shots_out',
 'team2_shots_on_target',
 'team2_shots_out',
 'team1_red_cards',
 'team2_red_cards',
 'team1_fouls',
 'team2_fouls',
 'team1_corners',
 'team2_corners',
 'team1_total_shots',
 'team2_total_shots',
 'date',
 'is_future_match',
 'goal_diff_team1',
 'goal_diff_team2',
 'corners_diff_team1',
 'corners_diff_team2',
 'team1_big_win',
 'team1_big_loss',
 'team2_big_win',
 'team2_big_loss',
 'team1_ah-2.5_win',
 'team1_ah-2.5_loss',
 'team2_ah-2.5_win',
 'team2_ah-2.5_loss',
 'team1_ah+2.5_win',
 'team1_ah+2.5_loss',
 'team2_ah+2.5_win',
 'team2_ah+2.5_loss',
 'team1_over4.5',
 'team1_under4.5',
 'team2_over4.5',
 'team2_under4.5',
 'team1_over3.5',
 'team1_under3.5',
 'team2_over3.5',
 'team2_under3.5',
 'team1_over6.5',
 'team1_under6.5',
 'team2_over6.5',
 'team2_under6.5',
 'team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_loss

In [6]:
columns_to_drop = [
 'team1_goals',
 'team2_goals',
 'team1_shots_on_target',
 'team1_shots_out',
 'team2_shots_on_target',
 'team2_shots_out',
 'team1_red_cards',
 'team2_red_cards',
 'team1_fouls',
 'team2_fouls',
 'team1_corners',
 'team2_corners',
 'team1_total_shots',
 'team2_total_shots',
 'date',
 'is_future_match',
 'goal_diff_team1',
 'goal_diff_team2',
 'corners_diff_team1',
 'corners_diff_team2',
 'team1_big_win',
 'team1_big_loss',
 'team2_big_win',
 'team2_big_loss',
 'team1_ah-2.5_win',
 'team1_ah-2.5_loss',
 'team2_ah-2.5_win',
 'team2_ah-2.5_loss',
 'team1_ah+2.5_win',
 'team1_ah+2.5_loss',
 'team2_ah+2.5_win',
 'team2_ah+2.5_loss',
 'team1_over4.5',
 'team1_under4.5',
 'team2_over4.5',
 'team2_under4.5',
 'team1_over3.5',
 'team1_under3.5',
 'team2_over3.5',
 'team2_under3.5',
 'team1_over6.5',
 'team1_under6.5',
 'team2_over6.5',
 'team2_under6.5',
 'result_team1',
 'result_team2']

Criando future Match baseado na data de hoje

In [7]:
dataset2 = combined_df_2023.copy()

# filter the DataFrame
future_matches = dataset2[dataset2['is_future_match'] == True]
future_matches.sort_values(by='date')
print(future_matches.shape)

from datetime import datetime, timedelta

# Get yesterday's date
yesterday = datetime.now() - timedelta(days=1)

# Filter the DataFrame to include only rows with dates greater than yesterday
#future_matches = future_matches[future_matches['date'] > yesterday]

# Liste todas as colunas que você deseja verificar
columns_to_check= [col for col in future_matches.columns if col not in columns_to_drop]

# Drop as linhas com 'np.nan' nas colunas especificadas
future_matches = future_matches.dropna(subset=columns_to_check)
future_matches_calculado = future_matches.drop(columns_to_drop,axis=1)
future_matches_calculado = future_matches_calculado.drop('season',axis=1)
"""# Contando os valores NaN em cada coluna
nan_counts = future_matches.isna().sum()

# Transformando em uma lista de pares (nome da coluna, contagem de np.nan)
nan_list = list(nan_counts.items())

# Percorrendo a lista e imprimindo cada valor individualmente com o nome da coluna
for col_name, nan_count in nan_list:
    print(f'{col_name}: {nan_count}')"""
    
print(future_matches_calculado.shape)

(360, 172)
(75, 125)


In [8]:
#CORTAR AS LINHAS COM MATCHES FUTUROS AQUI

# Replace empty strings with NaN
print(dataset2.shape)
dataset2.replace('', np.nan, inplace=True)
print(f"A quantidade de np.nan em linhas eram {dataset2.isna().sum().sum()}")
# Remove rows that contain any missing values
dataset2.dropna(inplace=True)

import numpy as np

counter = 0  # Initialize counter
for index, row in dataset2.iterrows():
    if row.isnull().any() or row.eq('').any():
        print(f"Row {index} contains 'NaN' or an empty value.")
        counter += 1  # Increase counter if condition is met

print(f"Total number of rows with 'NaN' or an empty value: {counter}")
dataset2.shape


(1182, 172)
A quantidade de np.nan em linhas eram 59010
Total number of rows with 'NaN' or an empty value: 0


(415, 172)

Concatenando os 2

In [9]:
datatotal = pd.concat([dataset, dataset2], ignore_index=True)#mudei o 'dataset' por combined_df_13c_new

datatotal.sort_values(by='date', inplace=True)

if 'team1_yellow_cards' in datatotal.columns:
    datatotal = datatotal.drop(['team1_yellow_cards'], axis=1)

if 'team2_yellow_cards' in datatotal.columns:
    datatotal = datatotal.drop(['team2_yellow_cards'], axis=1)

# Substituir valores maiores que 15 por 15 na coluna 'team1_corners'
datatotal.loc[datatotal['team1_corners'] > 15, 'team1_corners'] = 15

# Substituir valores maiores que 15 por 15 na coluna 'team2_corners'
datatotal.loc[datatotal['team2_corners'] > 15, 'team2_corners'] = 15    

datatotal

,championship,date,team1,team2,team1_goals,team2_goals,team1_total_shots,team2_total_shots,team1_shots_on_target,team2_shots_on_target,...,team1_winning_streak,team1_without_winning_streak,team2_losing_streak,team2_strength,team2_undefeated_streak,team2_winning_streak,team2_without_winning_streak,team1_suspended_players,team2_suspended_players,is_future_match
0,E0,2002-09-14,Charlton,Arsenal,0.0,3.0,9.0,10.0,3.0,8.0,...,0.0,1.0,0.0,0.666297,5.0,1.0,0.0,0.0,0.0,NaN
1,E0,2002-09-14,Everton,Middlesbrough,2.0,1.0,13.0,10.0,8.0,5.0,...,0.0,3.0,0.0,0.665927,1.0,1.0,0.0,0.0,0.0,NaN
2,E0,2002-09-14,Leeds,Man United,1.0,0.0,8.0,6.0,2.0,5.0,...,1.0,0.0,1.0,0.554939,0.0,0.0,1.0,0.0,0.0,NaN
3,E0,2002-09-14,West Brom,Southampton,1.0,0.0,11.0,10.0,7.0,5.0,...,2.0,0.0,0.0,0.332963,1.0,1.0,0.0,0.0,0.0,NaN
4,E0,2002-09-15,Man City,Blackburn,2.0,2.0,15.0,12.0,7.0,8.0,...,0.0,1.0,2.0,0.454133,0.0,0.0,3.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67252,SUE A,2023-09-18,aik,degerfors if,2.0,0.0,7.0,9.0,4.0,2.0,...,0.0,1.0,0.0,0.318794,1.0,1.0,0.0,0.0,0.0,False
67253,ESP B,2023-09-18,andorra cf,real oviedo,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,3.0,0.0,0.332779,2.0,0.0,5.0,0.0,0.0,False
67250,SUE A,2023-09-18,norrkoping,mjallby,0.0,2.0,5.0,9.0,1.0,4.0,...,0.0,2.0,0.0,0.478157,1.0,1.0,0.0,0.0,0.0,False
67251,BRA A,2023-09-18,bahia,santos,1.0,2.0,16.0,12.0,5.0,7.0,...,1.0,0.0,3.0,0.349942,0.0,0.0,3.0,0.0,0.0,False


In [10]:
champ_uniques = datatotal['championship'].unique()
print(champ_uniques)

['E0' 'SC0' 'E3' 'E2' 'E1' 'I1' 'SP1' 'D1' 'F1' 'P1' 'D2' 'SP2' 'T1' 'I2'
 'N1' 'F2' 'B1' 'G1' 'SUE A' 'BRA A' 'BEL A' 'ING D' 'ING C' 'ESP B'
 'ALE C' 'ALE B']


Converta Categorias em IDs Numéricos

Converta Categorias em IDs Numéricos

Converta Categorias em IDs Numéricos

Carregando a primeira vez

In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

# Suponho que 'datatotal' e 'future_matches_calculado' já estejam definidos e tenham as mesmas colunas relevantes

# Treinar o LabelEncoder com 'datatotal' e 'future_matches_calculado'
le_teams = LabelEncoder().fit(pd.concat([datatotal['team1'], datatotal['team2'], future_matches_calculado['team1'], future_matches_calculado['team2']]).astype(str))
le_champ = LabelEncoder().fit(pd.concat([datatotal['championship'], future_matches_calculado['championship']]).astype(str))

# Aplicar o LabelEncoder a 'datatotal'
datatotal['team1'] = le_teams.transform(datatotal['team1'].astype(str))
datatotal['team2'] = le_teams.transform(datatotal['team2'].astype(str))
datatotal['championship'] = le_champ.transform(datatotal['championship'].astype(str))

# Aplicar o mesmo LabelEncoder a 'future_matches_calculado'
future_matches_calculado['team1'] = le_teams.transform(future_matches_calculado['team1'].astype(str))
future_matches_calculado['team2'] = le_teams.transform(future_matches_calculado['team2'].astype(str))
future_matches_calculado['championship'] = le_champ.transform(future_matches_calculado['championship'].astype(str))

# Salvar o LabelEncoder
with open('le_teams.pkl', 'wb') as f:
    pickle.dump(le_teams, f)

# Salvar o le_champ
with open('le_champ.pkl', 'wb') as f:
    pickle.dump(le_champ, f)


Carregando os índices

Carregando os LabelEncoders Salvos e Aplicando aos Dados

In [11]:
import pickle

# Carregar o LabelEncoder para 'teams'
with open('le_teams.pkl', 'rb') as f:
    le_teams_loaded = pickle.load(f)

# Carregar o LabelEncoder para 'championship'
with open('le_champ.pkl', 'rb') as f:
    le_champ_loaded = pickle.load(f)

# Aplicar os LabelEncoders carregados aos seus dataframes
datatotal['team1'] = le_teams_loaded.transform(datatotal['team1'].astype(str))
datatotal['team2'] = le_teams_loaded.transform(datatotal['team2'].astype(str))
datatotal['championship'] = le_champ_loaded.transform(datatotal['championship'].astype(str))

future_matches_calculado['team1'] = le_teams_loaded.transform(future_matches_calculado['team1'].astype(str))
future_matches_calculado['team2'] = le_teams_loaded.transform(future_matches_calculado['team2'].astype(str))
future_matches_calculado['championship'] = le_champ_loaded.transform(future_matches_calculado['championship'].astype(str))


In [12]:
future_matches_calculado.head()

,team1,team2,championship,team1_big_wins_last5,team1_big_losses_last5,team2_big_wins_last5,team2_big_losses_last5,team1_ah-2.5_wins_last5,team1_ah-2.5_losses_last5,team2_ah-2.5_wins_last5,...,team2_winning_streak,team2_undefeated_streak,team2_losing_streak,team2_without_winning_streak,avg_points_lasts5_2,team2_strength,championship_points_2,rested_4_days_or_more_2,team1_suspended_players,team2_suspended_players
513,473,588,4,1.0,1.0,2.0,1.0,1.0,4.0,3.0,...,1.0,3.0,0.0,0.0,1.6,0.596039,1.695652,1.0,1.0,0.0
514,616,503,4,1.0,0.0,0.0,3.0,0.0,5.0,1.0,...,0.0,0.0,6.0,6.0,0.0,0.328074,0.666667,1.0,0.0,0.0
778,578,500,17,1.0,2.0,0.0,1.0,1.0,4.0,1.0,...,0.0,0.0,3.0,7.0,0.2,0.000000,0.142857,1.0,0.0,0.0
779,479,580,17,3.0,2.0,1.0,0.0,1.0,4.0,2.0,...,0.0,7.0,0.0,1.0,1.8,0.768640,1.857143,1.0,0.0,0.0
515,612,531,4,0.0,1.0,2.0,0.0,3.0,2.0,2.0,...,1.0,1.0,0.0,0.0,2.4,0.550908,1.521739,1.0,0.0,0.0


In [13]:
datatotal['season'] = datatotal['season'].astype('float64')


Separando X e Y

In [14]:
X = datatotal.drop(columns_to_drop, axis=1)
y1 = datatotal['team1_corners']
y2 = datatotal['team2_corners']
X.shape

(67255, 126)

In [15]:
list(X.dtypes)

[dtype('int32'),
 dtype('int32'),
 dtype('int32'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),

In [16]:
list(X.columns)

['championship',
 'team1',
 'team2',
 'season',
 'team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_losses_last5',
 'team1_ah-2.5_wins_last5',
 'team1_ah-2.5_losses_last5',
 'team2_ah-2.5_wins_last5',
 'team2_ah-2.5_losses_last5',
 'team1_ah+2.5_wins_last5',
 'team1_ah+2.5_losses_last5',
 'team2_ah+2.5_wins_last5',
 'team2_ah+2.5_losses_last5',
 'team1_over3.5_last5',
 'team1_under3.5_last5',
 'team2_over3.5_last5',
 'team2_under3.5_last5',
 'team1_over4.5_last5',
 'team1_under4.5_last5',
 'team2_over4.5_last5',
 'team2_under4.5_last5',
 'team1_over6.5_last5',
 'team1_under6.5_last5',
 'team2_over6.5_last5',
 'team2_under6.5_last5',
 'avg_scr_lasts3_1_home',
 'avg_scr_lasts5_1_home',
 'avg_scr_lasts3_1_away',
 'avg_scr_lasts5_1_away',
 'avg_conc_lasts3_1_home',
 'avg_conc_lasts5_1_home',
 'avg_conc_lasts3_1_away',
 'avg_conc_lasts5_1_away',
 'avg_scr_lasts3_2_home',
 'avg_scr_lasts5_2_home',
 'avg_scr_lasts3_2_away',
 'avg_scr_lasts5_2_away',
 'avg_

In [17]:
y1_uniques = datatotal['team1_corners'].unique()
print(y1_uniques)

[ 1. 10.  4.  0. 15.  9.  8.  5.  6.  7.  3. 12. 11. 14.  2. 13.]


In [18]:
# Contar ocorrências para 'y1'
y1_counts = y1.value_counts().sort_index().reindex(range(0, 31), fill_value=0)
print("Contagem de ocorrências para y1:")
print(y1_counts)

# Contar ocorrências para 'y2'
y2_counts = y2.value_counts().sort_index().reindex(range(0, 31), fill_value=0)
print("\nContagem de ocorrências para y2:")
print(y2_counts)


Contagem de ocorrências para y1:
team1_corners
0      698
1     2438
2     5033
3     7595
4     9224
5     9519
6     8643
7     7248
8     5585
9     3961
10    2782
11    1822
12    1116
13     712
14     417
15     462
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
Name: count, dtype: int64

Contagem de ocorrências para y2:
team2_corners
0      1562
1      4953
2      8163
3     10294
4     10663
5      9491
6      7400
7      5641
8      3587
9      2339
10     1371
11      850
12      416
13      283
14      137
15      105
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
30        0
Name: count, dtype: int64


Separando o treino e o teste e a normalização

In [19]:
from sklearn.preprocessing import StandardScaler

# Dividindo os dados com base na coluna 'season'
X_train = X[X['season'] < 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
X_test = X[X['season'] >= 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
y_train1 = y1[X['season'] < 2022]
y_test1 = y1[X['season'] >= 2022]
y_train2 = y2[X['season'] < 2022]
y_test2 = y2[X['season'] >= 2022]

# Escalando apenas as colunas que você quer (ajuste isso conforme suas necessidades)
cols_to_scale = [col for col in future_matches_calculado.columns if col not in ['team1', 'team2', 'championship']]

# Ajustar o escalonador com base no conjunto de treinamento
scaler = StandardScaler().fit(X_train[cols_to_scale])

# Reordenar as colunas para corresponder à ordem usada para ajustar o escalonador
X_train = X_train[cols_to_scale]
X_test = X_test[cols_to_scale]

# Aplicar o escalonamento
future_matches_calculado_scaled = future_matches_calculado.copy()
future_matches_calculado_scaled[cols_to_scale] = scaler.transform(future_matches_calculado[cols_to_scale])

# Transformar os conjuntos de treinamento e teste
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Preparar as colunas para o embedding
X_train_embed = X[X['season'] < 2022][['team1', 'team2', 'championship']]
X_test_embed = X[X['season'] >= 2022][['team1', 'team2', 'championship']]


In [20]:
list(X_train.columns)

['team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_losses_last5',
 'team1_ah-2.5_wins_last5',
 'team1_ah-2.5_losses_last5',
 'team2_ah-2.5_wins_last5',
 'team2_ah-2.5_losses_last5',
 'team1_ah+2.5_wins_last5',
 'team1_ah+2.5_losses_last5',
 'team2_ah+2.5_wins_last5',
 'team2_ah+2.5_losses_last5',
 'team1_over3.5_last5',
 'team1_under3.5_last5',
 'team2_over3.5_last5',
 'team2_under3.5_last5',
 'team1_over4.5_last5',
 'team1_under4.5_last5',
 'team2_over4.5_last5',
 'team2_under4.5_last5',
 'team1_over6.5_last5',
 'team1_under6.5_last5',
 'team2_over6.5_last5',
 'team2_under6.5_last5',
 'avg_scr_lasts3_1_home',
 'avg_scr_lasts5_1_home',
 'avg_scr_lasts3_1_away',
 'avg_scr_lasts5_1_away',
 'avg_conc_lasts3_1_home',
 'avg_conc_lasts5_1_home',
 'avg_conc_lasts3_1_away',
 'avg_conc_lasts5_1_away',
 'avg_scr_lasts3_2_home',
 'avg_scr_lasts5_2_home',
 'avg_scr_lasts3_2_away',
 'avg_scr_lasts5_2_away',
 'avg_conc_lasts3_2_home',
 'avg_conc_lasts5_2_home',


In [21]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_train2.shape)
print(y_test2.shape)
print(X_train_embed.shape)
print(X_test_embed.shape)
print(future_matches_calculado_scaled.shape)

(61353, 122)
(5902, 122)
(61353,)
(5902,)
(61353,)
(5902,)
(61353, 3)
(5902, 3)
(75, 125)


Concatenando ambos Ys em um único tensor

In [41]:
import numpy as np

y_train_combined = np.stack((y_train1, y_train2), axis=-1)
y_test_combined = np.stack((y_test1, y_test2), axis=-1)



def to_one_hot(difference):
    # Inicializa um vetor de zeros
    one_hot = np.zeros(11)
    
    if difference >= 5:
        one_hot[0] = 1
    elif difference == 4:
        one_hot[1] = 1
    elif difference == 3:
        one_hot[2] = 1
    elif difference == 2:
        one_hot[3] = 1
    elif difference == 1:
        one_hot[4] = 1
    elif difference == 0:
        one_hot[5] = 1
    elif difference == -1:
        one_hot[6] = 1
    elif difference == -2:
        one_hot[7] = 1
    elif difference == -3:
        one_hot[8] = 1
    elif difference == -4:
        one_hot[9] = 1
    elif difference <= -5:
        one_hot[10] = 1
    
    return one_hot

# Calcula as diferenças de placar entre y_train1 e y_train2
differences_train = y_train1 - y_train2
differences_teste = y_test1 - y_test2


# Converte as diferenças de placar para one-hot encoding
one_hot_train = np.array([to_one_hot(diff) for diff in differences_train])
one_hot_test = np.array([to_one_hot(diff) for diff in differences_teste])

print(one_hot_train.shape) 
print(one_hot_test.shape)

print(y_train_combined.shape)
print(y_test_combined.shape)

(61353, 11)
(5902, 11)
(61353, 2)
(5902, 2)


Função de perda

Backup

In [24]:
"""import tensorflow as tf

def custom_unified_loss(y_true, y_pred):
    # Separar y_true e y_pred para cada time
    y_true_team1, y_true_team2 = y_true[:, 0], y_true[:, 1]
    y_pred_team1, y_pred_team2 = y_pred[:, 0], y_pred[:, 1]
    
    # Calcular a média para cada time
    avg1 = tf.reduce_mean(y_true_team1)
    avg2 = tf.reduce_mean(y_true_team2)

    # Calcular o erro absoluto relativo para cada time
    normalized_error_team1 = tf.abs(y_true[:, 0] - y_pred[:, 0]) 
    normalized_error_team2 = tf.abs(y_true[:, 1] - y_pred[:, 1]) *(avg1/avg2)
    abs_error = (normalized_error_team1 + normalized_error_team2) / 2

    #_val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))
       # ...
    dif_prev = (y_pred_team1 - y_pred_team2)
    dif_true = (y_true_team1 - y_true_team2)
    
    # Calcula a diferença absoluta entre as diferenças previstas e verdadeiras
    abs_diff = tf.abs(dif_prev - dif_true)
    
    # Calcula um termo de bônus usando a função sigmoidal
    bonus_term = tf.math.tanh(10 * (1 - abs_diff))
    
    # Se a diferença é maior ou igual a 4 (ou menor ou igual a -4), aplica um fator negativo
    condition1 = tf.logical_and(dif_true >= 4, dif_prev >= 4)
    condition2 = tf.logical_and(dif_true <= -4, dif_prev <= -4)
    condition3 = abs_diff <= 1.5

    final_condition = tf.logical_or(tf.logical_or(condition1, condition2), condition3)

    abs_diff_acc = tf.where(final_condition, -1 * (abs_diff + bonus_term), abs_diff + bonus_term)


    # Calcular a diferença entre as previsões para as médias
    distance_to_avg1 = tf.abs(y_pred_team1 - avg1)
    distance_to_avg2 = tf.abs(y_pred_team2 - avg2)
    
    # Calcular os rewards
    reward1 = tf.math.log(distance_to_avg1 + 1)
    reward2 = tf.math.log(distance_to_avg2 + 1)

    # Calcular as penalties
    penalty1 = 1 * normalized_error_team1 / (distance_to_avg1 + 1)
    penalty2 = 1 * normalized_error_team2 / (distance_to_avg2 + 1)    

    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff1 = tf.math.tanh(penalty1 - reward1)
    normalized_diff2 = tf.math.tanh(penalty2 - reward2)
    normalized_diff = tf.math.tanh((normalized_diff1 + normalized_diff2)/2)

    # combined_error
    combined_error = abs_error + abs_diff_acc + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    combined_error_value = tf.maximum(combined_error, 0.0001)

    return tf.reduce_mean(combined_error_value)
"""

'import tensorflow as tf\n\ndef custom_unified_loss(y_true, y_pred):\n    # Separar y_true e y_pred para cada time\n    y_true_team1, y_true_team2 = y_true[:, 0], y_true[:, 1]\n    y_pred_team1, y_pred_team2 = y_pred[:, 0], y_pred[:, 1]\n    \n    # Calcular a média para cada time\n    avg1 = tf.reduce_mean(y_true_team1)\n    avg2 = tf.reduce_mean(y_true_team2)\n\n    # Calcular o erro absoluto relativo para cada time\n    normalized_error_team1 = tf.abs(y_true[:, 0] - y_pred[:, 0]) \n    normalized_error_team2 = tf.abs(y_true[:, 1] - y_pred[:, 1]) *(avg1/avg2)\n    abs_error = (normalized_error_team1 + normalized_error_team2) / 2\n\n    #_val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))\n       # ...\n    dif_prev = (y_pred_team1 - y_pred_team2)\n    dif_true = (y_true_team1 - y_true_team2)\n    \n    # Calcula a diferença absoluta entre as diferenças previstas e verdadeiras\n    abs_diff = tf.abs(dif_prev - dif_true)\n    \n 

Função de perda atual:

In [28]:
import tensorflow as tf

def calculate_median(tensor):
    sorted_tensor = tf.sort(tensor)
    mid = tf.shape(sorted_tensor)[0] // 2
    is_even = tf.equal(tf.shape(sorted_tensor)[0] % 2, 0)
    
    median = tf.cond(
        is_even,
        true_fn=lambda: (sorted_tensor[mid - 1] + sorted_tensor[mid]) / 2,
        false_fn=lambda: sorted_tensor[mid]
    )
    
    return median

def custom_unified_loss(y_true, y_pred):
    # Separar y_true e y_pred para cada time
    y_true_team1, y_true_team2 = y_true[:, 0], y_true[:, 1]
    y_pred_team1, y_pred_team2 = y_pred[:, 0], y_pred[:, 1]
    
    # Calcular a média para cada time
    avg1 = tf.reduce_mean(y_true_team1)
    avg2 = tf.reduce_mean(y_true_team2)

    # Calcular a mediana para cada time
    median1 = calculate_median(y_true_team1)
    median2 = calculate_median(y_true_team2)

    # Calcular o erro absoluto relativo para cada time
    normalized_error_team1 = tf.abs(y_true[:, 0] - y_pred[:, 0]) 
    normalized_error_team2 = tf.abs(y_true[:, 1] - y_pred[:, 1])
    abs_error = (normalized_error_team1 + normalized_error_team2) / 2 

    #_val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))
       # ...
    dif_prev = (y_pred_team1 - y_pred_team2)
    dif_true = (y_true_team1 - y_true_team2)
    
    # Calcula a diferença absoluta entre as diferenças previstas e verdadeiras
    abs_diff = tf.abs(dif_prev - dif_true)
    
    # Calcula um termo de bônus usando a função tanh
    bonus_term = tf.math.tanh(10 * (3.5 - abs_diff))
    
    # Se a diferença é maior ou igual a 4 (ou menor ou igual a -4), aplica um fator negativo
    condition1 = tf.logical_and(dif_true >= 3.5, dif_prev >= 3.5)
    condition2 = tf.logical_and(dif_true <= -3.5, dif_prev <= -3.5)
    condition3 = abs_diff <= 2

    final_condition = tf.logical_or(tf.logical_or(condition1, condition2), condition3)

    abs_diff_acc = tf.where(final_condition, -1 * (abs_diff + bonus_term), abs_diff + bonus_term)

    # Calcular a diferença entre as previsões para as médias
    distance_to_avg1 = tf.abs(y_pred_team1 - avg1)
    distance_to_avg2 = tf.abs(y_pred_team2 - avg2)

   # Calcular a diferença entre as previsões para as medianas
    distance_to_median1 = tf.abs(y_pred_team1 - median1)
    distance_to_median2 = tf.abs(y_pred_team2 - median2)

    # Calcular os rewards
    
    reward1 = tf.math.sigmoid(5 * (2.5- distance_to_median1))
    reward2 = tf.math.sigmoid(5 * (2.5- distance_to_median2))

    # Calcular as penalties
    penalty1 = 1 * normalized_error_team1 / (distance_to_avg1 + 1)
    penalty2 = 1 * normalized_error_team2 / (distance_to_avg2 + 1)    

    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff1 = tf.math.tanh(penalty1 - reward1)
    normalized_diff2 = tf.math.tanh(penalty2 - reward2)
    normalized_diff = tf.math.tanh((normalized_diff1 + normalized_diff2)/2)

    # combined_error
    combined_error = abs_error + 2 * abs_diff_acc + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    combined_error_value = tf.maximum(combined_error, 0.00001)

    return tf.reduce_mean(combined_error_value)


Testes de funções de perda

In [26]:
"""import numpy as np
import tensorflow as tf

def custom_unified_loss2(y_true, y_pred):
    # Separar y_true e y_pred para cada time
    y_true_team1, y_true_team2 = y_true[:, 0], y_true[:, 1]
    y_pred_team1, y_pred_team2 = y_pred[:, 0], y_pred[:, 1]
    
    # Calcular a média para cada time
    avg1 = 6.6
    avg2 = 4.6

    # Calcular o erro absoluto relativo para cada time
    normalized_error_team1 = tf.abs(y_true[:, 0] - y_pred[:, 0]) 
    normalized_error_team2 = tf.abs(y_true[:, 1] - y_pred[:, 1])
    abs_error = (normalized_error_team1 + normalized_error_team2)

    #_val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))
       # ...
    dif_prev = (y_pred_team1 - y_pred_team2)
    dif_true = (y_true_team1 - y_true_team2)
    
    # Calcula a diferença absoluta entre as diferenças previstas e verdadeiras
    abs_diff = tf.abs(dif_prev - dif_true)
    
    # Calcula um termo de bônus usando a função tanh
    bonus_term = tf.math.tanh(5 * (2.5- abs_diff))
    
    # Se a diferença é maior ou igual a 4 (ou menor ou igual a -4), aplica um fator negativo
    condition1 = tf.logical_and(dif_true >= 3.5, dif_prev >= 3.5)
    condition2 = tf.logical_and(dif_true <= -3.5, dif_prev <= -3.5)
    condition3 = abs_diff <= 1.5

    final_condition = tf.logical_or(tf.logical_or(condition1, condition2), condition3)

    abs_diff_acc = tf.where(final_condition, -1 * (abs_diff + bonus_term), abs_diff + bonus_term)

    # Calcular a diferença entre as previsões para as médias
    distance_to_avg1 = tf.abs(y_pred_team1 - avg1)
    distance_to_avg2 = tf.abs(y_pred_team2 - avg2)
    
    # Calcular os rewards
    reward1 = tf.math.log(distance_to_avg1 + 1)
    reward2 = tf.math.log(distance_to_avg2 + 1)

    # Calcular as penalties
    penalty1 = 1 * normalized_error_team1 / (distance_to_avg1 + 1)
    penalty2 = 1 * normalized_error_team2 / (distance_to_avg2 + 1)    

    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff1 = tf.math.tanh(penalty1 - reward1)
    normalized_diff2 = tf.math.tanh(penalty2 - reward2)
    normalized_diff = tf.math.tanh((normalized_diff1 + normalized_diff2))

    # Print rewards, penalties, and normalized_diff
    print(f"abs_error: {abs_error.numpy()}")
    print(f"reward1: {reward1.numpy()}, penalty1: {penalty1.numpy()}, normalized_diff1: {normalized_diff1.numpy()}")
    print(f"reward2: {reward2.numpy()}, penalty2: {penalty2.numpy()}, normalized_diff2: {normalized_diff2.numpy()}")
    print(f"normalized_diff: {normalized_diff.numpy()}")
    print(f"abs_diff {abs_diff.numpy()} | bonus_term {bonus_term.numpy()}--> abs_diff_acc: {abs_diff_acc.numpy()} ")  
    print(f"combined_error = abs_error: {abs_error.numpy()} + abs_diff_acc: {abs_diff_acc.numpy()} + normalized_diff: {normalized_diff.numpy()}")


    # combined_error
    combined_error = abs_error + abs_diff_acc + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    combined_error_value = tf.maximum(combined_error, 0.0001)

    return tf.reduce_mean(combined_error_value)

# Loop para gerar valores aleatórios e calcular a perda
for _ in range(20):
    # Gerando valores aleatórios para y_true e y_pred
    y_true_sample = np.random.randint(0, 10, size=(1, 2))
    y_pred_sample = np.random.randint(0, 10, size=(1, 2))
    
    # Convertendo para tensores do TensorFlow
    y_true_tensor = tf.convert_to_tensor(y_true_sample, dtype=tf.float32)
    y_pred_tensor = tf.convert_to_tensor(y_pred_sample, dtype=tf.float32)
    
    # Calculando a perda
    loss_value = custom_unified_loss2(y_true_tensor, y_pred_tensor).numpy()
    
    # Imprimindo os resultados
    print(f"y_true_team1: {y_true_sample[0][0]}, y_pred_team1: {y_pred_sample[0][0]}, y_true_team2: {y_true_sample[0][1]}, y_pred_team2: {y_pred_sample[0][1]}, Loss: {loss_value:.4f}\n")
"""

'import numpy as np\nimport tensorflow as tf\n\ndef custom_unified_loss2(y_true, y_pred):\n    # Separar y_true e y_pred para cada time\n    y_true_team1, y_true_team2 = y_true[:, 0], y_true[:, 1]\n    y_pred_team1, y_pred_team2 = y_pred[:, 0], y_pred[:, 1]\n    \n    # Calcular a média para cada time\n    avg1 = 6.6\n    avg2 = 4.6\n\n    # Calcular o erro absoluto relativo para cada time\n    normalized_error_team1 = tf.abs(y_true[:, 0] - y_pred[:, 0]) \n    normalized_error_team2 = tf.abs(y_true[:, 1] - y_pred[:, 1])\n    abs_error = (normalized_error_team1 + normalized_error_team2)\n\n    #_val_mae_diff = np.mean(np.abs((val_predict[:, 0] - val_predict[:, 1]) - (val_targ[:, 0] - val_targ[:, 1])))\n       # ...\n    dif_prev = (y_pred_team1 - y_pred_team2)\n    dif_true = (y_true_team1 - y_true_team2)\n    \n    # Calcula a diferença absoluta entre as diferenças previstas e verdadeiras\n    abs_diff = tf.abs(dif_prev - dif_true)\n    \n    # Calcula um termo de bônus usando a função

Arquitetura da NN

In [44]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam

# Obter o número de times e campeonatos únicos
n_teams = len(pd.concat([datatotal['team1'], datatotal['team2']]).unique())
n_champ = len(datatotal['championship'].unique())

# ==== Parte 1: Definindo os Inputs para o Embedding ====
# Estes são os inputs que vão alimentar os embeddings. 
# Cada input tem a dimensão de (1,) porque cada jogo tem exatamente um 'team1', um 'team2', e um 'championship'.
team1_input = Input(shape=(1,), name='Team1-Input')
team2_input = Input(shape=(1,), name='Team2-Input')
champ_input = Input(shape=(1,), name='Championship-Input')

# ==== Parte 2: Criando os Embeddings ====
# n_teams e n_champ são o número de times e campeonatos únicos, respectivamente.
# O output_dim é um hiperparâmetro para você ajustar. Ele define o tamanho do espaço de embedding.

# Embedding para o time 1
team1_embedding = Embedding(input_dim=n_teams, output_dim=50, name='Team1-Embedding')(team1_input)  # output_dim ajustável

# Embedding para o time 2
team2_embedding = Embedding(input_dim=n_teams, output_dim=50, name='Team2-Embedding')(team2_input)  # output_dim ajustável

# Embedding para o campeonato
champ_embedding = Embedding(input_dim=n_champ, output_dim=5, name='Championship-Embedding')(champ_input)  # output_dim ajustável

# ==== Parte 3: Achatando os Embeddings ====
# Cada embedding precisa ser achatado para ser concatenado posteriormente
team1_embedding = Flatten()(team1_embedding)
team2_embedding = Flatten()(team2_embedding)
champ_embedding = Flatten()(champ_embedding)

# ==== Parte 4: Outras Características ====
# Este é o input para as outras características (já escaladas) do seu conjunto de dados.
other_features_input = Input(shape=(X_train_scaled.shape[1],), name='Other-Features-Input')

# ==== Parte 5: Concatenando Tudo ====
# Aqui, todos os embeddings e as outras características são concatenados em um único vetor
merged = Concatenate()([team1_embedding, team2_embedding, champ_embedding, other_features_input])

# ==== Parte 6: Camadas Ocultas ====
# Estes são os neurônios e camadas totalmente conectadas (Dense) onde a "aprendizagem" real acontece.
# Você pode ajustar o número de neurônios, a função de ativação, e outros hiperparâmetros aqui.
from tensorflow.keras.regularizers import l1, l2

hidden_layer = Dense(512, activation='tanh', kernel_regularizer=l2(0.0001))(merged)
hidden_layer = Dropout(0.3)(hidden_layer)

hidden_layer = Dense(256, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(128, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(64, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(32, activation='tanh')(hidden_layer)


hidden_layer = Dense(16, activation='tanh')(hidden_layer)

hidden_layer = Dense(8, activation='tanh')(hidden_layer)

hidden_layer = Dense(4, activation='tanh')(hidden_layer)
# ==== Parte 7: Camada de Saída ====
# Esta é a camada de saída. A função de ativação 'linear' é usada para regressão.
output = Dense(11, activation='softmax', name='Output-Layer')(hidden_layer)

# ==== Parte 8: Compilando o Modelo ====
# Finalmente, o modelo é compilado. O otimizador Adam é usado, com uma taxa de aprendizagem de 0.001.
# A perda é definida como 'mean_squared_error', que é comum para problemas de regressão.
model = Model(
    inputs=[team1_input, team2_input, champ_input, other_features_input], 
    outputs=[output]
)


In [45]:
from tensorflow.keras.callbacks import EarlyStopping

# Parâmetros para Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# Número de épocas
n_epochs = 100

# Compilando o modelo
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento do modelo
history = model.fit(
    [X_train_embed['team1'], X_train_embed['team2'], X_train_embed['championship'], X_train_scaled], 
    one_hot_train,  # Usando os vetores one-hot como rótulos
    epochs=n_epochs,
    batch_size=380,
    verbose=1,
    validation_data=([X_test_embed['team1'], X_test_embed['team2'], X_test_embed['championship'], X_test_scaled], one_hot_test),  # Usando os vetores one-hot como rótulos
    callbacks=[early_stopping]
)


Epoch 1/100
162/162 [==============================] - 4s 16ms/step - loss: 2.4259 - accuracy: 0.1038 - val_loss: 2.3787 - val_accuracy: 0.1301
Epoch 2/100
162/162 [==============================] - 2s 13ms/step - loss: 2.3860 - accuracy: 0.1515 - val_loss: 2.3539 - val_accuracy: 0.2253
Epoch 3/100
162/162 [==============================] - 2s 12ms/step - loss: 2.3610 - accuracy: 0.2073 - val_loss: 2.3392 - val_accuracy: 0.2309
Epoch 4/100
162/162 [==============================] - 2s 12ms/step - loss: 2.3453 - accuracy: 0.2200 - val_loss: 2.3293 - val_accuracy: 0.2314
Epoch 5/100
162/162 [==============================] - 2s 13ms/step - loss: 2.3340 - accuracy: 0.2231 - val_loss: 2.3198 - val_accuracy: 0.2314
Epoch 6/100
162/162 [==============================] - 2s 12ms/step - loss: 2.3260 - accuracy: 0.2235 - val_loss: 2.3136 - val_accuracy: 0.2316
Epoch 7/100
162/162 [==============================] - 2s 12ms/step - loss: 2.3201 - accuracy: 0.2243 - val_loss: 2.3102 - val_accuracy:

Salvando o modelo

In [50]:
# Salvar o modelo
model.save('modelo17-09-512neu-softmax.keras')

Carregando o modelo

In [51]:
from tensorflow.keras.models import load_model

# Substitua 'caminho/para/sua/pasta/' pelo caminho real onde o modelo está salvo
caminho_completo_para_modelo = r'C:\Users\mathe\OneDrive\Área de Trabalho\SoccerIA\MathIA_v7\modelo17-09-512neu-softmax.keras'

# Carregar o modelo
model = load_model(caminho_completo_para_modelo)


In [34]:
future_matches_calculado_scaled.shape

(75, 125)

In [35]:
future_matches_calculado_scaled.head()

,team1,team2,championship,team1_big_wins_last5,team1_big_losses_last5,team2_big_wins_last5,team2_big_losses_last5,team1_ah-2.5_wins_last5,team1_ah-2.5_losses_last5,team2_ah-2.5_wins_last5,...,team2_winning_streak,team2_undefeated_streak,team2_losing_streak,team2_without_winning_streak,avg_points_lasts5_2,team2_strength,championship_points_2,rested_4_days_or_more_2,team1_suspended_players,team2_suspended_players
513,473,588,4,0.174966,0.123596,1.174694,0.184672,-0.333442,0.333442,1.415019,...,0.208571,0.240951,-0.525751,-0.690139,0.322560,1.243029,1.100764,0.443706,2.587813,-0.305174
514,616,503,4,0.174966,-0.986648,-0.939662,2.454561,-1.263209,1.263209,-0.405417,...,-0.569612,-0.661135,5.144209,1.562600,-2.132589,-2.098684,-2.303008,0.443706,-0.334677,-0.305174
778,578,500,17,0.174966,1.233839,-0.939662,0.184672,-0.333442,0.333442,-0.405417,...,-0.569612,-0.661135,2.309229,1.938057,-1.825696,-6.189989,-4.035713,0.443706,-0.334677,-0.305174
779,479,580,17,2.339654,1.233839,0.117516,-0.950273,-0.333442,0.333442,0.504801,...,-0.569612,1.443734,-0.525751,-0.314683,0.629454,3.395473,1.634958,0.443706,-0.334677,-0.305174
515,612,531,4,-0.907378,0.123596,1.174694,-0.950273,1.526092,-1.526092,0.504801,...,0.208571,-0.360440,-0.525751,-0.690139,1.550135,0.680212,0.525479,0.443706,-0.334677,-0.305174


Output!!!!!

In [55]:
# Inicializando uma lista vazia para armazenar as previsões
corner_predictions = []

# Iterar sobre cada linha em 'future_matches' e 'future_matches_calculado_scaled'
for (index1, row1), (index2, row2) in zip(future_matches.iterrows(), future_matches_calculado_scaled.iterrows()):
    # Prepare os dados de entrada para a previsão
    team1_input_data = np.array([[row2['team1']]], dtype=np.float32)
    team2_input_data = np.array([[row2['team2']]], dtype=np.float32)
    champ_input_data = np.array([[row2['championship']]], dtype=np.float32)
    other_features_data = np.array([row2.drop(['team1', 'team2', 'championship']).astype(np.float32)])
    
    # Faça a previsão usando o modelo
    pred = model.predict([team1_input_data, team2_input_data, champ_input_data, other_features_data])
    
    # Convertendo as previsões (que são arrays numpy) para listas para melhor visualização
    pred_list = pred[0].tolist()
    
    # Armazenar as previsões na lista
    corner_predictions.append([row1['date'], row1['championship'], row1['team1'], row1['team2'],  pred_list])
    
    # Imprimir as previsões
    print(f"Prediction for match between {row1['team1']} and {row1['team2']}: {pred_list}")

# Criar um DataFrame a partir da lista de previsões
df = pd.DataFrame(corner_predictions, columns=['Date', 'Championship', 'Team1_name', 'Team2_name', 'Match_prediction'])

df


1/1 [==============================] - 0s 25ms/step
Prediction for match between américa-mg and red bull bragantino: [0.06108884513378143, 0.035069841891527176, 0.047015395015478134, 0.06396017968654633, 0.07434239983558655, 0.08537810295820236, 0.10987068712711334, 0.11711212247610092, 0.08700147271156311, 0.07535058259963989, 0.24381034076213837]
1/1 [==============================] - 0s 25ms/step
Prediction for match between vasco da gama and coritiba: [0.2286933958530426, 0.09753315150737762, 0.10212413221597672, 0.1056770533323288, 0.09757782518863678, 0.09902679920196533, 0.06989987194538116, 0.05674178525805473, 0.052825093269348145, 0.03692718967795372, 0.05297376215457916]
1/1 [==============================] - 0s 28ms/step
Prediction for match between peterborough united and cheltenham town: [0.3722003698348999, 0.09183824807405472, 0.08767630904912949, 0.0922001376748085, 0.08098390698432922, 0.0741356909275055, 0.06122023984789848, 0.049952182918787, 0.03739139437675476, 0.

,Date,Championship,Team1_name,Team2_name,Match_prediction
0,2023-09-19,BRA A,américa-mg,red bull bragantino,"[0.06108884513378143, 0.035069841891527176, 0...."
1,2023-09-19,BRA A,vasco da gama,coritiba,"[0.2286933958530426, 0.09753315150737762, 0.10..."
2,2023-09-19,ING C,peterborough united,cheltenham town,"[0.3722003698348999, 0.09183824807405472, 0.08..."
3,2023-09-19,ING C,barnsley fc,portsmouth,"[0.09984815120697021, 0.05387510731816292, 0.0..."
4,2023-09-20,BRA A,são paulo,fortaleza,"[0.21739892661571503, 0.08649764209985733, 0.1..."
...,...,...,...,...,...
70,2023-09-24,SUE A,mjallby,ifk gotemburgo,"[0.17892128229141235, 0.0959566980600357, 0.11..."
71,2023-09-25,SUE A,ifk varnamo,ik sirius,"[0.18424934148788452, 0.08140559494495392, 0.1..."
72,2023-09-25,SUE A,halmstad,elfsborg,"[0.06474359333515167, 0.03956413269042969, 0.0..."
73,2023-09-25,ESP B,racing de ferrol,real saragoza,"[0.16573555767536163, 0.06847675889730453, 0.0..."


In [60]:
# Inicializando uma lista vazia para armazenar as odds
odds_predictions = []

# Iterar sobre cada linha no DataFrame de previsões
for index, row in df.iterrows():
    pred_probs = row['Match_prediction']
    
    # Calcular as probabilidades para cada mercado
    #4.5
    team1_minus45_prob = pred_probs[0]
    team1_plus45_prob = 1 - pred_probs[10]
    team2_minus45_prob = pred_probs[10]
    team2_plus45_prob = 1 - pred_probs[0]

    #3.5
    team1_minus35_prob = pred_probs[0] + pred_probs[1]
    team1_plus35_prob = 1 - pred_probs[10] - pred_probs[9]
    team2_minus35_prob = pred_probs[10] + pred_probs[9]
    team2_plus35_prob = 1 - pred_probs[0] - pred_probs[1]
    #2.5
    team1_minus25_prob = pred_probs[0] + pred_probs[1] + pred_probs[2]
    team1_plus25_prob = 1 - pred_probs[10] - pred_probs[9] - pred_probs[8]
    team2_minus25_prob = pred_probs[10] + pred_probs[9] + pred_probs[8]
    team2_plus25_prob = 1 - pred_probs[0] - pred_probs[1] - pred_probs[2]
    #1.5
    team1_minus15_prob = pred_probs[0] + pred_probs[1] + pred_probs[2] + pred_probs[3]
    team1_plus15_prob = 1 - pred_probs[10] - pred_probs[9] - pred_probs[8] - pred_probs[7]
    team2_minus15_prob = pred_probs[10] + pred_probs[9] + pred_probs[8] + pred_probs[7]
    team2_plus15_prob = 1 - pred_probs[0] - pred_probs[1] - pred_probs[2] - pred_probs[3]
    #0.5
    team1_minus05_prob = pred_probs[0] + pred_probs[1]+ pred_probs[2] + pred_probs[3] + pred_probs[4]
    team1_plus05_prob = 1 - pred_probs[10] - pred_probs[9] - pred_probs[8] - pred_probs[7]- pred_probs[6]
    team2_minus05_prob = pred_probs[10] + pred_probs[9] + pred_probs[8] + pred_probs[7] + pred_probs[6]
    team2_plus05_prob = 1 - pred_probs[0] - pred_probs[1] - pred_probs[2] - pred_probs[3]- pred_probs[4]

    # Converter as probabilidades em odds
    team1_minus45_odds = 1 / team1_minus45_prob
    team1_plus45_odds = 1 / team1_plus45_prob
    team1_minus35_odds = 1 / team1_minus35_prob
    team1_plus35_odds = 1 / team1_plus35_prob
    team1_minus25_odds = 1 / team1_minus25_prob
    team1_plus25_odds = 1 / team1_plus25_prob
    team1_minus15_odds = 1 / team1_minus15_prob
    team1_plus15_odds = 1 / team1_plus15_prob
    team1_minus05_odds = 1 / team1_minus05_prob
    team1_plus05_odds = 1 / team1_plus05_prob
    team2_minus45_odds = 1 / team2_minus45_prob
    team2_plus45_odds = 1 / team2_plus45_prob
    team2_minus35_odds = 1 / team2_minus35_prob
    team2_plus35_odds = 1 / team2_plus35_prob
    team2_minus25_odds = 1 / team2_minus25_prob
    team2_plus25_odds = 1 / team2_plus25_prob
    team2_minus15_odds = 1 / team2_minus15_prob
    team2_plus15_odds = 1 / team2_plus15_prob
    team2_minus05_odds = 1 / team2_minus05_prob
    team2_plus05_odds = 1 / team2_plus05_prob
    
    # Armazenar as odds na lista
    odds_predictions.append([
        row['Date'], row['Championship'],row['Team1_name'], row['Team2_name'], 
        team1_minus45_odds, team1_plus45_odds,
        team1_minus35_odds, team1_plus35_odds,
        team1_minus25_odds, team1_plus25_odds,
        team1_minus15_odds, team1_plus15_odds,
        team1_minus05_odds, team1_plus05_odds,
        team2_minus45_odds, team2_plus45_odds,
        team2_minus35_odds, team2_plus35_odds,
        team2_minus25_odds, team2_plus25_odds,
        team2_minus15_odds, team2_plus15_odds,
        team2_minus05_odds, team2_plus05_odds
    ])

# Criar um DataFrame a partir da lista de odds
odds_df = pd.DataFrame(odds_predictions, columns=[
     'Date', 'Championship','Team1_name', 'Team2_name',
    'Team1_-4.5_Odds', 'Team1_+4.5_Odds',
    'Team1_-3.5_Odds', 'Team1_+3.5_Odds',
    'Team1_-2.5_Odds', 'Team1_+2.5_Odds',
    'Team1_-1.5_Odds', 'Team1_+1.5_Odds',
    'Team1_-0.5_Odds', 'Team1_+0.5_Odds',
    'Team2_-4.5_Odds', 'Team2_+4.5_Odds',
    'Team2_-3.5_Odds', 'Team2_+3.5_Odds',
    'Team2_-2.5_Odds', 'Team2_+2.5_Odds',
    'Team2_-1.5_Odds', 'Team2_+1.5_Odds',
    'Team2_-0.5_Odds', 'Team2_+0.5_Odds'
])

# Converte a coluna 'date' para o tipo de data do pandas
odds_df['Date'] = pd.to_datetime(odds_df['Date'])

# Formata a coluna 'date' para o formato de data brasileiro (dd/mm/yyyy)
odds_df['Date'] = odds_df['Date'].dt.strftime('%d/%m/%Y')

odds_df = odds_df.sort_values(['Championship', 'Date'])

odds_df

,Date,Championship,Team1_name,Team2_name,Team1_-4.5_Odds,Team1_+4.5_Odds,Team1_-3.5_Odds,Team1_+3.5_Odds,Team1_-2.5_Odds,Team1_+2.5_Odds,...,Team2_-4.5_Odds,Team2_+4.5_Odds,Team2_-3.5_Odds,Team2_+3.5_Odds,Team2_-2.5_Odds,Team2_+2.5_Odds,Team2_-1.5_Odds,Team2_+1.5_Odds,Team2_-0.5_Odds,Team2_+0.5_Odds
16,22/09/2023,ALE B,osnabruck,hamburgo sv,22.096005,1.350162,12.122736,1.531088,7.766737,1.795209,...,3.855825,1.047402,2.882927,1.089906,2.257532,1.147782,1.818115,1.239270,1.541981,1.355619
19,22/09/2023,ALE B,fc magdeburg,sc paderborn,3.493196,1.038222,2.592729,1.071539,2.058417,1.129277,...,27.163084,1.401092,14.978400,1.627853,8.735306,1.944807,5.993608,2.446009,4.369474,3.178185
38,23/09/2023,ALE B,fc st. pauli,schalke 04,2.568321,1.031568,2.167970,1.053913,1.829813,1.094382,...,32.677149,1.637624,19.548375,1.856187,11.595292,2.205091,7.091786,2.729436,4.822065,3.636373
50,23/09/2023,ALE B,greuther furth,karlsruher sc,7.705151,1.133444,5.097740,1.216307,3.668511,1.346016,...,8.493788,1.149139,5.623067,1.244037,3.890044,1.374741,2.876239,1.577182,2.268276,1.851611
51,23/09/2023,ALE B,eintracht braunschweig,fc nuremberga,11.209328,1.168097,7.443554,1.289113,5.131964,1.482330,...,6.948948,1.097950,4.458857,1.155194,3.073269,1.242016,2.255702,1.387055,1.843765,1.597743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,24/09/2023,SUE A,kalmar,bk hacken,11.612503,1.127492,7.541525,1.253613,4.747761,1.415659,...,8.843635,1.094228,4.943012,1.152870,3.405819,1.266826,2.558389,1.435620,2.072761,1.741298
60,24/09/2023,SUE A,varbergs bois fc,hammarby if,21.114063,1.297606,11.149906,1.476092,7.190986,1.751084,...,4.360146,1.049716,3.100434,1.098523,2.331409,1.161525,1.868028,1.265640,1.591766,1.391819
70,24/09/2023,SUE A,mjallby,ifk gotemburgo,5.589050,1.068893,3.637978,1.121814,2.588984,1.192917,...,15.515189,1.217910,9.209239,1.379078,6.183568,1.629333,4.644443,1.971032,3.477489,2.490259
71,25/09/2023,SUE A,ifk varnamo,ik sirius,5.427428,1.082841,3.764282,1.136895,2.730326,1.211202,...,13.071280,1.225865,8.304851,1.361758,5.734802,1.577926,4.206196,1.871493,3.121213,2.332144


In [58]:
odds_df.to_excel("teste_df_softmax.xlsx")

Criando e tratando os times para verificar jogos próximos

In [37]:
lista_teams = list(odds_df['Team1_name']) + list(odds_df['Team2_name']) 

In [38]:
lista_teams

['flamengo',
 'internacional',
 'red bull bragantino',
 'coritiba',
 'fortaleza',
 'santos',
 'fc nuremberga',
 'sc paderborn',
 'salford city',
 'real saragoza',
 'palmeiras',
 'cuiabá',
 'kvc westerlo',
 'msv duisburgo',
 'bradford city',
 'vasco da gama',
 'albacete',
 'real valladolid',
 'forest green rovers',
 'accrington stanley',
 'afc wimbledon',
 'fleetwood town',
 'huesca',
 'gillingham',
 'wrexham',
 'stevenage f.c.',
 'reading',
 'shrewsbury town',
 'newport county afc',
 'mk dons',
 'atlético mineiro',
 'crawley town',
 'port vale fc',
 'rot-weiss essen',
 'peterborough united',
 'karlsruher sc',
 'wycombe wanderers',
 'derby county',
 'barnsley fc',
 'lincoln city',
 'clube brugge',
 'rwd molenbeek 47',
 'royal union saint-gilloise',
 'hansa rostock',
 'ifk gotemburgo',
 'exeter city',
 'djurgardens if',
 'sv sandhausen',
 'sv elversberg',
 'vfb lubeck',
 'bielefeld',
 'schalke 04',
 'ik sirius',
 'borussia dortmund ii',
 'fc ingolstadt',
 'wigan fc',
 'eibar',
 'hertha b

Printando as diferenças de datas

In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from datetime import datetime

# Dicionário para converter o nome do mês de português para inglês
month_map = {
    'jan': 'Jan',
    'fev': 'Feb',
    'mar': 'Mar',
    'abr': 'Apr',
    'mai': 'May',
    'jun': 'Jun',
    'jul': 'Jul',
    'ago': 'Aug',
    'set': 'Sep',
    'out': 'Oct',
    'nov': 'Nov',
    'dez': 'Dec'
}

# Função para extrair a data do texto do confronto
def extract_date(confronto_text):
    date_match = re.search(r'(\d{1,2}) (set|out|jan|fev|mar|abr|mai|jun|jul|ago|nov|dez)', confronto_text.lower())
    if date_match:
        day = date_match.group(1)
        month_pt = date_match.group(2)
        month_en = month_map.get(month_pt)
        date_str = f"{day} {month_en}"
        return datetime.strptime(date_str, '%d %b')
    return None

# Restante do código segue o mesmo.


def is_relevant_game(confronto_text):
    irrelevant_keywords = ['Femin', 'Sub', 'women', 'u20']
    for keyword in irrelevant_keywords:
        if keyword.lower() in confronto_text.lower():
            return False
    return True

def colored_print(text, days):
    color_code = "\033[91m" if days <= 4 else "\033[0m"  # 91 é o código para vermelho
    print(f"{text} {color_code}{days}\033[0m")  # \033[0m reseta a cor

with open('saida_dif_dias.txt', 'w') as f:
    for time in lista_teams:
        try:
            driver = webdriver.Chrome()
            driver.maximize_window()
            driver.get('https://oddspedia.com/br/futebol')
            
            wait = WebDriverWait(driver, 10)
            search_box = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div/input')))
            
            search_box.send_keys(time)
            search_box.send_keys(Keys.RETURN)
            
            all_matches = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div[2]/ul/li/div')))[:7]
            
            filtered_matches = [match for match in all_matches if is_relevant_game(match.text)]
            
            filtered_dates = [extract_date(match.text) for match in filtered_matches if extract_date(match.text)]

            f.write(f"\nPróximas partidas para {time}:\n")
            print(f"\nPróximas partidas para {time}:")
            
            for i, match in enumerate(filtered_matches):
                output = f"\nJogo {i+1}:\n{match.text}\n"  # Conteúdo do jogo
                f.write(output)  # Escreve no arquivo
                print(output)  # Imprime na tela
                
                if i < len(filtered_dates) - 1:
                    diff = (filtered_dates[i + 1] - filtered_dates[i]).days
                    colored_output = f"Diferença de dias para o próximo jogo: {diff}\n"
                    f.write(colored_output)  # Escreve no arquivo
                    colored_print("Diferença de dias para o próximo jogo:", diff)  # Imprime na tela
            
            f.write("-"*70 + "\n")  # Escreve no arquivo
            print("-"*70)
            
        except Exception as e:
            error_output = f"Erro: {str(e)}\n"
            f.write(error_output)  # Escreve no arquivo
            print(error_output)  # Imprime na tela
            import traceback
            traceback.print_exc()
        
        finally:
            driver.close()









Próximas partidas para flamengo:

Jogo 1:
BRASILEIRÃO SÉRIE A
13 set
Flamengo
Athletico-PR

Diferença de dias para o próximo jogo: 4

Jogo 2:
COPA DO BRASIL
17 set
Flamengo
São Paulo

Diferença de dias para o próximo jogo: 3

Jogo 3:
BRASILEIRÃO SÉRIE A
20 set
Goiás
Flamengo

Diferença de dias para o próximo jogo: 4

Jogo 4:
COPA DO BRASIL
24 set
São Paulo
Flamengo

Diferença de dias para o próximo jogo: 7

Jogo 5:
BRASILEIRÃO SÉRIE A
01 out
Flamengo
Bahia

----------------------------------------------------------------------

Próximas partidas para internacional:

Jogo 1:
LIGA DRUGA CRNE GORE
13 set
FK Lovcen
FK Internacional Podgorica

Diferença de dias para o próximo jogo: 0

Jogo 2:
BRASILEIRÃO SÉRIE A
13 set
Internacional
São Paulo

Diferença de dias para o próximo jogo: 3

Jogo 3:
SINGAPURA PRIMEIRA LIGA
16 set
Home United
Geylang Internacional

Diferença de dias para o próximo jogo: 1

Jogo 4:
LIGA DRUGA CRNE GORE
17 set
FK Internacional Podgorica
FK Iskra Danilovgrad

Diferen

Traceback (most recent call last):
  File "C:\Users\mathe\AppData\Local\Temp\ipykernel_3088\3735419640.py", line 63, in <module>
    all_matches = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div[2]/ul/li/div')))[:7]
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 




Próximas partidas para fc nuremberga:

Jogo 1:
LIGA REGIONAL BAYERN
15 set
Greuther Furth II
1 FC Nuremberga II

Diferença de dias para o próximo jogo: 0

Jogo 2:
2ª BUNDESLIGA
15 set
FC Nuremberga
Greuther Furth

Diferença de dias para o próximo jogo: 8

Jogo 3:
2ª BUNDESLIGA
23 set
Eintracht Braunschweig
FC Nuremberga

Diferença de dias para o próximo jogo: 0

Jogo 4:
LIGA REGIONAL BAYERN
23 set
1 FC Nuremberga II
Eintracht Bamberg 2010

----------------------------------------------------------------------

Próximas partidas para sc paderborn:

Jogo 1:
2ª BUNDESLIGA
15 set
SC Paderborn
Wehen Wiesbaden

Diferença de dias para o próximo jogo: 7

Jogo 2:
2ª BUNDESLIGA
22 set
FC Magdeburg
SC Paderborn

Diferença de dias para o próximo jogo: 0

Jogo 3:
REGIONALLIGA OESTE
22 set
Monchengladbach II
SC Paderborn 07 II

----------------------------------------------------------------------

Próximas partidas para salford city:

Jogo 1:
LEAGUE TWO
15 set
Salford City
Notts County

Diferença 

Traceback (most recent call last):
  File "C:\Users\mathe\AppData\Local\Temp\ipykernel_3088\3735419640.py", line 63, in <module>
    all_matches = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div[2]/ul/li/div')))[:7]
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 




Próximas partidas para kvc westerlo:

Jogo 1:
PRIMEIRA DIVISÃO A
15 set
KVC Westerlo
Royal Antwerp FC

Diferença de dias para o próximo jogo: 7

Jogo 2:
PRIMEIRA DIVISÃO A
22 set
Standard Liege
KVC Westerlo

Diferença de dias para o próximo jogo: 9

Jogo 3:
PRIMEIRA DIVISÃO A
01 out
Genk
KVC Westerlo

----------------------------------------------------------------------

Próximas partidas para msv duisburgo:

Jogo 1:
3. LIGA
15 set
MSV Duisburgo
SC Verl

Diferença de dias para o próximo jogo: 8

Jogo 2:
3. LIGA
23 set
Köln
MSV Duisburgo

Diferença de dias para o próximo jogo: 4

Jogo 3:
3. LIGA
27 set
Borussia Dortmund II
MSV Duisburgo

Diferença de dias para o próximo jogo: 3

Jogo 4:
3. LIGA
30 set
MSV Duisburgo
Preusen Munster

----------------------------------------------------------------------

Próximas partidas para bradford city:

Jogo 1:
LEAGUE TWO
16 set
Bradford City
Harrogate Town FC

Diferença de dias para o próximo jogo: 7

Jogo 2:
LEAGUE TWO
23 set
Newport County AFC


...


Próximas partidas para osnabruck:

Jogo 1:
2ª BUNDESLIGA
17 set
Hannover Sportverein
Osnabruck

Diferença de dias para o próximo jogo: 5

Jogo 2:
2ª BUNDESLIGA
22 set
Osnabruck
Hamburgo SV

----------------------------------------------------------------------

Próximas partidas para standard liege:

Jogo 1:
PRIMEIRA DIVISÃO A
17 set
Eupen
Standard Liege

Diferença de dias para o próximo jogo: 5

Jogo 2:
PRIMEIRA DIVISÃO A
22 set
Standard Liege
KVC Westerlo

Diferença de dias para o próximo jogo: 8

Jogo 3:
PRIMEIRA DIVISÃO A
30 set
Oud-Heverlee Leuven
Standard Liege

Diferença de dias para o próximo jogo: 8

Jogo 4:
PRIMEIRA DIVISÃO A
08 out
Standard Liege
Clube Brugge

Diferença de dias para o próximo jogo: 13

Jogo 5:
PRIMEIRA DIVISÃO A
21 out
Standard Liege
Anderlecht

----------------------------------------------------------------------
Erro: Message: 




Traceback (most recent call last):
  File "C:\Users\mathe\AppData\Local\Temp\ipykernel_3088\3735419640.py", line 63, in <module>
    all_matches = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div[2]/ul/li/div')))[:7]
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 




Próximas partidas para anderlecht:

Jogo 1:
PRIMEIRA DIVISÃO A
17 set
KV Kortrijk
Anderlecht

Diferença de dias para o próximo jogo: 7

Jogo 2:
PRIMEIRA DIVISÃO A
24 set
Anderlecht
Clube Brugge

Diferença de dias para o próximo jogo: 6

Jogo 3:
PRIMEIRA DIVISÃO A
30 set
Eupen
Anderlecht

----------------------------------------------------------------------
Erro: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A5BB52A2+57122]
	(No symbol) [0x00007FF7A5B2EA92]
	(No symbol) [0x00007FF7A59FE3AB]
	(No symbol) [0x00007FF7A5A37D3E]
	(No symbol) [0x00007FF7A5A37E2C]
	(No symbol) [0x00007FF7A5A70B67]
	(No symbol) [0x00007FF7A5A5701F]
	(No symbol) [0x00007FF7A5A6EB82]
	(No symbol) [0x00007FF7A5A56DB3]
	(No symbol) [0x00007FF7A5A2D2B1]
	(No symbol) [0x00007FF7A5A2E494]
	GetHandleVerifier [0x00007FF7A5E5EF82+2849794]
	GetHandleVerifier [0x00007FF7A5EB1D24+3189156]
	GetHandleVerifier [0x00007FF7A5EAACAF+3160367]
	GetHandleVerifier [0x00007FF7A5C46D06+653702]
	(No symbol) [0x00007FF7A5B3A208]


Traceback (most recent call last):
  File "C:\Users\mathe\AppData\Local\Temp\ipykernel_3088\3735419640.py", line 58, in <module>
    search_box = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/aside/div[1]/div/input')))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A5BB52A2+57122]
	(No symbol) [0x00007FF7A5B2EA92]
	(No symbol) [0x00007FF7A59FE3AB]
	(No symbol) [0x00007FF7A5A37D3E]
	(No symbol) [0x00007FF7A5A37E2C]
	(No symbol) [0x00007FF7A5A70B67]
	(No symbol) [0x00007FF7A5A5701F]
	(No symbol) [0x00007FF7A5A6EB82]
	(No symbol) [0x00007FF7A5A56DB3]
	(No sym


Próximas partidas para halmstad:

Jogo 1:
LIGA ALLSVENSKAN
17 set
BK Hacken
Halmstad

Diferença de dias para o próximo jogo: 8

Jogo 2:
LIGA ALLSVENSKAN
25 set
Halmstad
Elfsborg

Diferença de dias para o próximo jogo: 5

Jogo 3:
LIGA ALLSVENSKAN
30 set
Djurgardens IF
Halmstad

----------------------------------------------------------------------

Próximas partidas para holstein kiel:

Jogo 1:
LIGA REGIONAL NORTE
13 set
Holstein Kiel II
Meppen

Diferença de dias para o próximo jogo: 4

Jogo 2:
LIGA REGIONAL NORTE
17 set
Holstein Kiel II
SC Spelle-Venhaus

Diferença de dias para o próximo jogo: 0

Jogo 3:
2ª BUNDESLIGA
17 set
FC St. Pauli
Holstein Kiel

Diferença de dias para o próximo jogo: 6

Jogo 4:
LIGA REGIONAL NORTE
23 set
Holstein Kiel II
Bremer SV

Diferença de dias para o próximo jogo: 1

Jogo 5:
2ª BUNDESLIGA
24 set
Holstein Kiel
Hertha Berlim

----------------------------------------------------------------------

Próximas partidas para ssv ulm 1846:

Jogo 1:
3. LIGA
17 set
